In [1]:
# What version of Python do you have?
import sys
import os
import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf
import numpy as np
import re
import mne
import pathlib
import openpyxl
from datetime import datetime
import pytz
import matplotlib
import random
import os
from skimage.restoration import (denoise_wavelet, estimate_sigma)
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, Sequential
from matplotlib import pyplot as plt
from scipy import signal
plt.ioff()
import psutil
import gc
import glob
pd.io.parquet.get_engine('auto').__class__
%matplotlib inline
matplotlib.use('Qt5Agg')
#matplotlib.use('agg')

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Init Plugin
Init Graph Optimizer
Init Kernel
Tensor Flow Version: 2.5.0
Keras Version: 2.5.0

Python 3.9.7 (default, Sep 16 2021, 23:53:23) 
[Clang 12.0.0 ]
Pandas 1.3.1
Scikit-Learn 0.24.2
GPU is available


In [2]:
cwd = os.getcwd()
database_path = '../../Dataset/EMU_monitor(ruc)/'
save_csv_path = '../../Dataset/EMU_monitor(ruc)/NHR/'
edf_file_type = ".edf"
capitilize_edf_file_type = ".EDF"
bdf_file_type = ".BDF"
patient_one_path = 'chb04/'
info_df_path = "../../Dataset/EMU_monitor(ruc)/Eventlist_RUC.xlsx"
external_hardisk_drive_path = os.path.dirname('/Volumes/LaCie/Database/')
cwd

'/Users/niklashjort/Desktop/Notes/Speciale/projects/DataHandling/Køge'

In [3]:
info_df = pd.read_excel(info_df_path, sheet_name="NHR_EDF")

In [4]:
info_df.head()

,patientID,time_emu,SeizureID,delay,seizureDuration,fileName
0,4,22-05-2019 02:45:22,1,83.000001,56,20190521155643_p4Nat2.sdeeg_converted_.easy_fi...
1,4,2019-05-22 03:50:49,2,83.000000,53,20190521155643_p4Nat2.sdeeg_converted_.easy_fi...
2,4,2019-05-22 05:27:56,3,81.000000,48,20190521155643_p4Nat2.sdeeg_converted_.easy_fi...
3,4,2019-05-22 07:07:04,4,81.000000,52,20190521155643_p4Nat2.sdeeg_converted_.easy_fi...
4,4,2019-05-22 07:51:24,5,81.000000,53,20190521155643_p4Nat2.sdeeg_converted_.easy_fi...


In [5]:
for i, r in info_df.iterrows():
    patient_id = r['patientID']
    patient_folder = "Patient " + str(patient_id)
    EEG_file = r['fileName']
    for folder in os.listdir(database_path + "EEG"):
        if patient_folder == folder and EEG_file != 0:
            full_path_patient_file = database_path + f"EEG/{patient_folder}/{EEG_file}"
            info_df.loc[i, "fullPath"] = full_path_patient_file

In [6]:
info_list = []

for i, r in info_df.iterrows():
    patient_id = r['patientID']
    patient_file = r['fullPath']
    container = {"ID": patient_id, "File": patient_file}

    if container not in info_list:
        info_list.append(container)

In [7]:
info_list[0:5]

[{'ID': 4,
  'File': '../../Dataset/EMU_monitor(ruc)/EEG/Patient 4/20190521155643_p4Nat2.sdeeg_converted_.easy_filtered.edf'},
 {'ID': 4,
  'File': '../../Dataset/EMU_monitor(ruc)/EEG/Patient 4/20190522090132_p4dag3.sdeeg_converted_.easy_filtered.edf'},
 {'ID': 6,
  'File': '../../Dataset/EMU_monitor(ruc)/EEG/Patient 6/20190826155114_p6nat1.sdeeg_converted_.easy_filtered.edf'},
 {'ID': 6,
  'File': '../../Dataset/EMU_monitor(ruc)/EEG/Patient 6/20190829094329_p6dag4.sdeeg_converted_.easy_filtered.edf'},
 {'ID': 24,
  'File': '../../Dataset/EMU_monitor(ruc)/EEG/Patient 24/ANONYMOU.BDF'}]

In [8]:
class container():
    def __init__(self, delay, time_emu, duration, id):
        self.delay = delay
        self.time_emu = time_emu
        self.duration = duration
        self.id = id

In [9]:
file_sz_info = []

for c in info_list:
    f = c['File']
    p = c['ID']
    print(f)
    cont_storage = []
    sz_count = 0
    for i, r in info_df.iterrows():
        if f == r['fullPath']:
            con = container(delay=r['delay'], time_emu=r['time_emu'], duration=r['seizureDuration'], id=r['SeizureID'])
            #cont = {"delay": r['delay'], "time_emu": r['time_emu'], "duration": r['seizureDuration'], "sz_id": r['SeizureID']}
            cont_storage.append(con)
    file_sz_info.append([p, f, cont_storage])

../../Dataset/EMU_monitor(ruc)/EEG/Patient 4/20190521155643_p4Nat2.sdeeg_converted_.easy_filtered.edf
../../Dataset/EMU_monitor(ruc)/EEG/Patient 4/20190522090132_p4dag3.sdeeg_converted_.easy_filtered.edf
../../Dataset/EMU_monitor(ruc)/EEG/Patient 6/20190826155114_p6nat1.sdeeg_converted_.easy_filtered.edf
../../Dataset/EMU_monitor(ruc)/EEG/Patient 6/20190829094329_p6dag4.sdeeg_converted_.easy_filtered.edf
../../Dataset/EMU_monitor(ruc)/EEG/Patient 24/ANONYMOU.BDF
../../Dataset/EMU_monitor(ruc)/EEG/Patient 28/ANONYMOU.BDF
../../Dataset/EMU_monitor(ruc)/EEG/Patient 30/P30_TrackIT.edf
../../Dataset/EMU_monitor(ruc)/EEG/Patient 21/TRACKIT_R1.BDF
../../Dataset/EMU_monitor(ruc)/EEG/Patient 21/TRACKIT_R2.BDF
../../Dataset/EMU_monitor(ruc)/EEG/Patient 21/TRACKIT_R3.BDF
../../Dataset/EMU_monitor(ruc)/EEG/Patient 21/TRACKIT_R4.BDF
../../Dataset/EMU_monitor(ruc)/EEG/Patient 33/ANONYMOU.BDF
../../Dataset/EMU_monitor(ruc)/EEG/Patient 34/ANONYMOU.BDF
../../Dataset/EMU_monitor(ruc)/EEG/Patient 35/ANON

In [10]:
def ReadEdfFile(file_name, print_reader_info = False):
    if edf_file_type in file_name or capitilize_edf_file_type in file_name:
        if(print_reader_info):
            data = mne.io.read_raw_edf(file_name)
            raw_data = data.get_data()
            converted_raw = pd.DataFrame(raw_data.transpose(), columns=data.ch_names)
            print(data.info)
            return converted_raw, data.info
        else:
            data = mne.io.read_raw_edf(file_name, verbose='error')
            raw_data = data.get_data()
            converted_raw = pd.DataFrame(raw_data.transpose(), columns=data.ch_names)
            return converted_raw, data.info
    if bdf_file_type in file_name:
        if(print_reader_info):
            data = mne.io.read_raw_bdf(file_name)
            raw_data = data.get_data()
            converted_raw = pd.DataFrame(raw_data.transpose(), columns=data.ch_names)
            print(data.info)
            return converted_raw, data.info
        else:
            data = mne.io.read_raw_edf(file_name, verbose='error')
            raw_data = data.get_data()
            converted_raw = pd.DataFrame(raw_data.transpose(), columns=data.ch_names)
            return converted_raw, data.info

In [11]:
def convert_date_to_ms(date_time):
    date_time = str(date_time)
    if "+" in str(date_time):
        date_time = str(date_time).split("+")[0]

    try:
        timestamp_ms = datetime.strptime(date_time, '%Y-%m-%d %H:%M:%S').timestamp() * 1000
    except:
        timestamp_ms = datetime.strptime(date_time, '%d-%m-%Y %H:%M:%S').timestamp() * 1000
    return timestamp_ms

In [12]:
def insert_time_stamp(dataframe, file_start_time, frq):
    timestamp_ms = convert_date_to_ms(file_start_time)
    period_row_increment_value =  (1 / int(frq)) * 1000
    dataframe.insert(0, "timestamp", [timestamp_ms + i * period_row_increment_value for i in dataframe.index])

In [13]:
def insert_class_col(dataframe, sz_info_list, freq):
    print(f"modtaget string: {sz_info_list}")
    
    if "class" not in dataframe.columns:
        dataframe.insert(0, "class", np.nan)

    if len(sz_info_list) == 0:
        dataframe.loc[(dataframe['class'] != "Seizure") & (dataframe['class'] != "Preictal I") & (dataframe['class'] != "Preictal II"), "class"] = "Interictal"
    else:
        for container in sz_info_list:
            delay = container.delay * 1000
            duration = container.duration * 1000
            sz_start = convert_date_to_ms(container.time_emu) + delay
            sz_end = sz_start + duration
            print(f"sz_start index = {sz_start}")
            print(f"sz_end: {sz_end}")
            preictal_start = sz_start - (15 * 60 * 1000)
            dataframe['timestamp'] = pd.to_numeric(dataframe['timestamp'])

            #INSERTING PREICTAL
            dataframe.loc[(dataframe['class'] != "seizure") & (dataframe['timestamp'] >= preictal_start) & (dataframe['timestamp'] < sz_start), "class"] = "Preictal"

            #INSERTING SEIZURE CLASS
            dataframe.loc[(dataframe['timestamp'] >= sz_start) & (dataframe['timestamp'] < sz_end), "class"] = "seizure"

            #INSERTING INTERICTAL
            dataframe.loc[(dataframe['class'] != "seizure") & (dataframe['class'] != "Preictal"), "class"] = "Interictal"

            print(dataframe["class"].value_counts())
    gc.collect()

In [19]:
def logging_info_txt(csv_file_name, freq, channels):
    file_object = open(save_csv_path + "info.txt", "a")
    file_object.write(f"\nfilename: {csv_file_name} \n freq: {freq} \n channels: {channels} \n")
    file_object.close()

In [20]:
for e in file_sz_info[0:1]:
    print(f"patient_id: {e[0]}")
    print(f"file_name: {e[1]}")

    file_path = e[1]
    df, data_info = ReadEdfFile(file_path)
    file_sample_rate = data_info["sfreq"]
    file_meas_date = data_info["meas_date"]
    file_channel = data_info['ch_names']
    relevant_channels = file_channel[0:2]
    print(f"freq: {file_sample_rate} meas: {file_meas_date} channels: {relevant_channels}")
    
    insert_time_stamp(df, file_meas_date, file_sample_rate)
    insert_class_col(df, e[2], file_sample_rate)

    save_file_name = f"patient_{e[0]}_date_{file_meas_date}.csv"

    #SAVE TO CSV

    #LOGGING:
    logging_info_txt(save_file_name, file_sample_rate, file_channel)

    #Memory:
    del df, data_info
    gc.collect()
    


patient_id: 4
file_name: ../../Dataset/EMU_monitor(ruc)/EEG/Patient 4/20190521155643_p4Nat2.sdeeg_converted_.easy_filtered.edf
freq: 500.0 meas: 2019-05-21 15:56:43+00:00 channels: ['CH-1', 'CH-2']
modtaget string: [<__main__.container object at 0x128432e20>, <__main__.container object at 0x29d1d8ee0>, <__main__.container object at 0x12849d2b0>, <__main__.container object at 0x29b092910>, <__main__.container object at 0x12849ad90>, <__main__.container object at 0x29d1de250>]
sz_start index = 1558486005000.0005
sz_end: 1558486061000.0005
Interictal    31922000
Preictal        450000
seizure          28000
Name: class, dtype: int64
sz_start index = 1558489932000.0
sz_end: 1558489985000.0
Interictal    31445500
Preictal        900000
seizure          54500
Name: class, dtype: int64
sz_start index = 1558495757000.0
sz_end: 1558495805000.0
Interictal    30971500
Preictal       1350000
seizure          78500
Name: class, dtype: int64
sz_start index = 1558501705000.0
sz_end: 1558501757000.0
I

In [16]:
df.head()

,class,timestamp,CH-1,CH-2,CH-3,CH-4,CH-5,CH-6,CH-7,CH-8,x,y,z
0,Interictal,1.558447e+12,0.000017,0.000058,0.001171,0.001181,-0.000045,-0.000051,-0.000065,0.000008,8.610537e-08,3.069114e-08,4.796582e-08
1,Interictal,1.558447e+12,0.000011,0.000053,0.001170,0.001180,-0.000051,-0.000057,-0.000070,0.000003,8.610537e-08,3.069114e-08,4.796582e-08
2,Interictal,1.558447e+12,0.000036,0.000079,0.001169,0.001180,-0.000026,-0.000032,-0.000045,0.000029,8.610537e-08,3.069114e-08,4.796582e-08
3,Interictal,1.558447e+12,0.000038,0.000079,0.001168,0.001179,-0.000025,-0.000032,-0.000045,0.000029,8.610537e-08,3.069114e-08,4.796582e-08
4,Interictal,1.558447e+12,0.000064,0.000105,0.001167,0.001178,0.000003,-0.000005,-0.000017,0.000056,8.610537e-08,3.069114e-08,4.796582e-08


In [17]:
# Skal ikke bruge nedenstående midlertidigt...
def get_file_t_start_ms(filename):
    str_t_start = re.match(r".*Patient [0-9]+/(.*?)_", filename)
    if str_t_start:
        str_t_start = str_t_start.group(1)
        t_start = datetime.strptime(str_t_start, '%Y%m%d%H%M%S')
        return t_start.timestamp() * 1000
    else:
        str_t_start = "Time start is not present"
        return str_t_start
    

In [18]:
datetime.fromtimestamp(1558495757000.0/1000)

datetime.datetime(2019, 5, 22, 5, 29, 17)